In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyarrow

     |████████████████████████████████| 17.7MB 50.7MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tliavgg4
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-tliavgg4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 42.7MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1285604 sha256=47be350b9d17fd40d03b27fa44c43ebdb69f5f875efab2eb5bd9cc047389c4bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-ro3ya5rv/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=26ac22994d0d0367dfbc84439

In [ ]:
# Check that we have a GPU
!nvidia-smi

Sat Oct 31 18:04:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=32_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
    max_seq_length=200,
)

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-cased')# (config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.num_parameters()

108340804

In [ ]:
!pwd

/content


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased", truncation = True, padding=True, max_length=200)

In [ ]:
cd "/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work"

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./tenk_join_8k.txt",
    block_size=100,
)

CPU times: user 3min 14s, sys: 6.64 s, total: 3min 20s
Wall time: 2min 12s


In [ ]:
!pwd

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
!pwd

/content/drive/My Drive/praveen_iiit_hyd_ire_class/ten_k_project_work


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tenkBERTv2Cased",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
#del [trainer]
#gc.collect()
#torch.cuda.empty_cache()

In [ ]:
%%time
trainer.train()

Step,Training Loss
500,4.971075
1000,4.559712
1500,4.400961
2000,4.233240
2500,4.089010
3000,4.071043
3500,3.967018
4000,3.899508
4500,3.863074
5000,3.765777


CPU times: user 4h 42min 56s, sys: 3h 26min 30s, total: 8h 9min 27s
Wall time: 8h 11min 38s


TrainOutput(global_step=48558, training_loss=3.1042982747436056)

In [ ]:
trainer.save_model("./tenkBERTv2Cased")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./tenkBERTv2Cased",
    tokenizer = tokenizer
)

Some weights of BertModel were not initialized from the model checkpoint at ./tenkBERTv2Cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
fill_mask("Registrant [MASK] definitive best knowledge information.")

[{'score': 0.1677379608154297,
  'sequence': '[CLS] Registrant regarding definitive best knowledge information. [SEP]',
  'token': 4423,
  'token_str': 'regarding'},
 {'score': 0.10854171961545944,
  'sequence': '[CLS] Registrant reference definitive best knowledge information. [SEP]',
  'token': 3835,
  'token_str': 'reference'},
 {'score': 0.10196356475353241,
  'sequence': '[CLS] Registrant best definitive best knowledge information. [SEP]',
  'token': 1436,
  'token_str': 'best'},
 {'score': 0.08150218427181244,
  'sequence': '[CLS] Registrant com definitive best knowledge information. [SEP]',
  'token': 3254,
  'token_str': 'com'},
 {'score': 0.05401173606514931,
  'sequence': '[CLS] Registrant knowledge definitive best knowledge information. [SEP]',
  'token': 3044,
  'token_str': 'knowledge'}]

In [ ]:
fill_mask("filed Section Act [MASK] Securities required.")

[{'score': 0.5749001502990723,
  'sequence': '[CLS] filed Section Act 1934 Securities required. [SEP]',
  'token': 3729,
  'token_str': '1934'},
 {'score': 0.062213387340307236,
  'sequence': '[CLS] filed Section Act 1933 Securities required. [SEP]',
  'token': 3698,
  'token_str': '1933'},
 {'score': 0.051575034856796265,
  'sequence': '[CLS] filed Section Act 405 Securities required. [SEP]',
  'token': 26904,
  'token_str': '405'},
 {'score': 0.050765905529260635,
  'sequence': '[CLS] filed Section Act statement Securities required. [SEP]',
  'token': 4195,
  'token_str': 'statement'},
 {'score': 0.04269832745194435,
  'sequence': '[CLS] filed Section Act Commission Securities required. [SEP]',
  'token': 2827,
  'token_str': 'Commission'}]

In [ ]:
fill_mask("Reports [MASK] Statements 8-K  Form.")

[{'score': 0.9265838265419006,
  'sequence': '[CLS] Reports Financial Statements 8 - K Form. [SEP]',
  'token': 7748,
  'token_str': 'Financial'},
 {'score': 0.010293220169842243,
  'sequence': '[CLS] Reports Current Statements 8 - K Form. [SEP]',
  'token': 9493,
  'token_str': 'Current'},
 {'score': 0.008259247988462448,
  'sequence': '[CLS] Reports Consolidated Statements 8 - K Form. [SEP]',
  'token': 23582,
  'token_str': 'Consolidated'},
 {'score': 0.005152004770934582,
  'sequence': '[CLS] Reports Index Statements 8 - K Form. [SEP]',
  'token': 10146,
  'token_str': 'Index'},
 {'score': 0.0046477061696350574,
  'sequence': '[CLS] Reports Annual Statements 8 - K Form. [SEP]',
  'token': 8451,
  'token_str': 'Annual'}]